Import required libraries.

In [ ]:
import pydub
import numpy as np
import sounddevice as sd
from matplotlib import pyplot as plt
from scipy.io.wavfile import read, write

These are my own implemented functions. 'dotted' outputs 1.5 times the duration, and getDurations outputs the amount of samples in an array so as to be used later.
Indices are:
    Whole note = 0
    Dotted whole note = 1
    Half note = 2
    Dotted half note = 3
    Quarter note = 4
    Dotted quarter note = 5
    etc.

In [16]:
def getDurations(tempo, fs):
    whole_note = 4*60*fs/tempo
    durs = np.array([[whole_note/(2**i),dotted(whole_note/(2**i))] for i in range(6)])
    
    return durs.flatten()

def dotted(dur):
    return dur*1.5

These are the functions from helper.ipynb but edited so as to implement tempo into the mix.

In [26]:
def sin_wave(f, n, fs):
    x = np.linspace(0, 2*np.pi, n)
    xp = np.linspace(0, -1*(n*ring/fs), n)
    y = np.sin(x*f*(n/fs))*np.exp(xp)
    z = np.zeros([n, 2])
    z[:, 0] = y
    z[:, 1] = y
    return z

def play_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]), fs)
    sd.play(y, fs)
    sd.wait()
    return 

def put_note(note_id, octave, dur, fs):
    if (note_id < 3) :
        octave += 1
    y = sin_wave(notes_base[note_id]*2**octave, int(notes_duration[dur]), fs)
    return y

def get_music(music_notes, fs):
    m = []
    for item in music_notes:
        y = put_note(item[0], item[1], item[2], fs)
        m.append(y)
    m = np.concatenate(m, 0)
    return m

This is a testbench. We set A4 = 440 Hz and the tempo and ringing values for the sine wave, get the note duration array from the tempo and then proceed to play the riff and the C Major scale.

In [40]:
notes_base = 2**(np.arange(12)/12)*27.5 # A0; A4= 440 Hz
tempo = 150
ring = 7 # how long the sine wave rings. lower value = more ringing/sustain
fs = 44100
notes_duration = getDurations(tempo,fs)
"""
    Whole note = 0
    Dotted whole note = 1
    Half note = 2
    Dotted half note = 3
    Quarter note = 4
    Dotted quarter note = 5
    etc.
"""
notes_ann = ['A', 'A#', 'B', 'C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#']

Scale = [[3,4,4], [5,4,4], [7,4,4], [8,4,4], [10,4,4], [0,4,4], [2,4,4], [3,5,4], 
        [2,4,4], [0,4,4], [10,4,4], [8,4,4], [7,4,4], [5,4,4], [3,4,4]]
theRiff = [ [5,4,6], [7,4,6], [8,4,6], [10,4,6], [7,4,4], [3,4,6], [5,4,2] ]

y = get_music(theRiff, fs)
sd.play(y, fs)

Exporting the results to .wav files.

In [41]:
write("the riff.wav", fs, y.astype(np.float32))
write("C Major scale.wav", fs, get_music(theRiff, fs).astype(np.float32))